###### 1. Read ANd Select  Data


In [57]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types 
from pyspark.sql.functions import col
from pyspark.sql.functions import when
from pyspark.sql.types import StringType
from pyspark.sql.functions import split , count



In [58]:

spark = SparkSession \
        .builder \
        .appName("asses") \
        .getOrCreate()

In [59]:
df1= spark.read.options(header='True', inferSchema='True', delimiter=',') \
  .csv("./asses/2022-03/*-outcomes.csv") \
  .withColumn("districtName", F.input_file_name())

In [60]:
df1.printSchema()

root
 |-- Crime ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Reported by: string (nullable = true)
 |-- Falls within: string (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- LSOA code: string (nullable = true)
 |-- LSOA name: string (nullable = true)
 |-- Outcome type: string (nullable = true)
 |-- districtName: string (nullable = false)



In [61]:
df2= spark.read.options(header='True', inferSchema='True', delimiter=',') \
  .csv("./asses/2022-03/*-street.csv") \
  .withColumn("districtName", F.input_file_name())

In [62]:
df2.printSchema()

root
 |-- Crime ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Reported by: string (nullable = true)
 |-- Falls within: string (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- LSOA code: string (nullable = true)
 |-- LSOA name: string (nullable = true)
 |-- Crime type: string (nullable = true)
 |-- Last outcome category: string (nullable = true)
 |-- Context: string (nullable = true)
 |-- districtName: string (nullable = false)



In [93]:
df2.show(6)

+--------------------+-------+--------------------+--------------------+---------+---------+--------------------+---------+-------------------+--------------------+---------------------+-------+--------------------+
|            Crime ID|  Month|         Reported by|        Falls within|Longitude| Latitude|            Location|LSOA code|          LSOA name|          Crime type|Last outcome category|Context|        districtName|
+--------------------+-------+--------------------+--------------------+---------+---------+--------------------+---------+-------------------+--------------------+---------------------+-------+--------------------+
|6b1be5a8275fcda29...|2022-03|Metropolitan Poli...|Metropolitan Poli...|-0.445898|50.803304|On or near Lamorn...|E01031422|          Arun 008B|Violence and sexu...|  Under investigation|   null|file:/home/brig/a...|
|6b1be5a8275fcda29...|2022-03|Metropolitan Poli...|Metropolitan Poli...|-0.445898|50.803304|On or near Lamorn...|E01031422|          Aru

In [ ]:
outcomes = df1.select(col("Crime ID").alias("CrimeID"),
                      col("districtName"),
                      col("Outcome type").alias("outcomeType"),
                      col("Latitude"),
                      col("Longitude")
                     )

In [110]:
outcomes.show(10)

+--------------------+------------+---------------+---------+---------+
|             CrimeID|districtName|    outcomeType| Latitude|Longitude|
+--------------------+------------+---------------+---------+---------+
|c026deed0f8c1fbfe...|metropolitan|Suspect charged|51.494629|-0.224336|
|a91407d7616943d13...|metropolitan|Suspect charged|51.517858|-0.246538|
|097f3199e65b4d2fa...|metropolitan|Suspect charged|  51.4987|-0.214079|
|a70d8667129c59595...|metropolitan|Suspect charged|51.519281|-0.200388|
|577df8e02358a0d3c...|metropolitan|Suspect charged|51.491112|-0.150388|
|52b59b0978b87cbe9...|metropolitan|Suspect charged|51.513732|-0.162068|
|49c33aae4ddcc4652...|metropolitan|Suspect charged|51.502048|-0.216642|
|c1104525f837da6a6...|metropolitan|Suspect charged|51.515453|-0.134932|
|4f464638a7678ab23...|metropolitan|Suspect charged|51.502989|-0.226461|
|1a7a1aba66ca20be5...|metropolitan|Suspect charged| 51.50045|-0.181679|
+--------------------+------------+---------------+---------+---

In [86]:

street = df2.select(col("Crime ID").alias("CrimeID"), 
                    col("districtName"),
                    col("Latitude"),
                    col("Longitude"),
                    col("Crime type").alias("crimeType"),
                    col("Last outcome category").alias("lastOutcomeCategory")
                    )

In [87]:
street.show(10)

+--------------------+--------------------+---------+---------+--------------------+--------------------+
|             CrimeID|        districtName| Latitude|Longitude|           crimeType| lastOutcomeCategory|
+--------------------+--------------------+---------+---------+--------------------+--------------------+
|6b1be5a8275fcda29...|file:/home/brig/a...|50.803304|-0.445898|Violence and sexu...| Under investigation|
|6b1be5a8275fcda29...|file:/home/brig/a...|50.803304|-0.445898|Violence and sexu...| Under investigation|
|b466ed98f60835f37...|file:/home/brig/a...|50.804178|-0.530681|Violence and sexu...| Under investigation|
|cc667a5d8c2fa4f30...|file:/home/brig/a...|51.137084| 0.876572|Theft from the pe...|Investigation com...|
|a60d9d1cff047a67e...|file:/home/brig/a...|51.124173| 0.969845|Violence and sexu...| Under investigation|
|62823a01dfebac344...|file:/home/brig/a...|51.809402|-0.813313|Violence and sexu...| Under investigation|
|62823a01dfebac344...|file:/home/brig/a...|51.

###### 1. Align and join data


In [88]:
street = street.withColumn('districtName', split(col('districtName'), '-')[3])




In [95]:
outcomes = outcomes.withColumn('districtName', split(col('districtName'), '-')[3])


In [96]:
outcomes.select(col('districtName')).distinct().show()

+---------------+
|   districtName|
+---------------+
|   metropolitan|
|     lancashire|
|      hampshire|
|     merseyside|
|           west|
|          essex|
|         thames|
|           kent|
|     humberside|
|          south|
| leicestershire|
|    northumbria|
|         sussex|
|nottinghamshire|
|       cheshire|
|           avon|
|      cleveland|
|     derbyshire|
|  hertfordshire|
| cambridgeshire|
+---------------+
only showing top 20 rows



In [97]:
street.select(col('districtName')).distinct().show()

+---------------+
|   districtName|
+---------------+
|           west|
|   metropolitan|
|     lancashire|
|      hampshire|
|     merseyside|
|         thames|
|          south|
|           avon|
|          essex|
|    northumbria|
|       northern|
|           kent|
| leicestershire|
|         sussex|
|     derbyshire|
|nottinghamshire|
|       cheshire|
| cambridgeshire|
|      cleveland|
|          devon|
+---------------+
only showing top 20 rows



In [98]:
street.distinct().show(1115)


+--------------------+------------+---------+---------+--------------------+--------------------+
|             CrimeID|districtName| Latitude|Longitude|           crimeType| lastOutcomeCategory|
+--------------------+------------+---------+---------+--------------------+--------------------+
|f4818b555bbbd0cf8...|metropolitan|51.582311| 0.140192|Criminal damage a...| Under investigation|
|307998477ae1c5207...|metropolitan|51.589112| 0.140035|Violence and sexu...| Under investigation|
|1ca44b83eaf540d44...|metropolitan|51.563135| 0.178464|            Burglary| Under investigation|
|42bf0d19f10a1ea0b...|metropolitan|51.551164|  0.16253|               Drugs| Under investigation|
|16a31ca5bab76d8cb...|metropolitan|51.534088| 0.145297|            Burglary| Under investigation|
|7076f4c80189953fa...|metropolitan|51.540119|  0.08057|             Robbery| Under investigation|
|                null|metropolitan|51.541889|  0.08779|Anti-social behav...|                null|
|ad4ec42d892450d31..

In [99]:
outcomes.distinct().show(1115)

+--------------------+------------+--------------------+---------+---------+
|             CrimeID|districtName|         outcomeType| Latitude|Longitude|
+--------------------+------------+--------------------+---------+---------+
|64d0c2ee59136bcb2...|metropolitan|     Suspect charged|51.538149|-0.260077|
|2f55e4563d741d34b...|metropolitan|     Suspect charged|51.529895|-0.104051|
|9d82aa0e23953c4f1...|metropolitan|     Suspect charged|51.468059|-0.366132|
|d0b1d639b9f896b5f...|metropolitan|     Suspect charged|51.489218|-0.141448|
|77137bec8cb707f4d...|metropolitan|     Suspect charged|51.475783|-0.047398|
|6af6302aa3b6ec6f4...|metropolitan|     Suspect charged|51.378891| 0.104492|
|fc650be2f27f58d62...|metropolitan|     Suspect charged| 51.48154|-0.076317|
|93b914bb2d51f9a1a...|metropolitan|     Suspect charged|51.439622|-0.126684|
|b18d9209a7cbe62c5...|metropolitan|     Suspect charged|51.491944|-0.100022|
|e2f49394a0481a08f...|metropolitan|     Suspect charged|51.466261|-0.102803|

In [103]:
fin = street.join(outcomes,["CrimeID", "districtName", "latitude", "longitude"], "inner") \
           .select("crimeID",
                              "districtName",
                              "latitude",
                              "longitude",
                              "crimeType",
                              "lastOutcomeCategory"                              
                             )

                      
                


In [104]:
fin.show(10)
fin.printSchema()

+--------------------+--------------+---------+---------+--------------------+--------------------+
|             crimeID|  districtName| latitude|longitude|           crimeType| lastOutcomeCategory|
+--------------------+--------------+---------+---------+--------------------+--------------------+
|001174e6e58476b4e...|  metropolitan| 51.50706|-0.140419|Theft from the pe...|Investigation com...|
|0013210e621a0f1ae...|         essex|51.663367| 0.079879|Violence and sexu...|Investigation com...|
|00140cba13b2a068e...|  metropolitan|51.558589|-0.280243|Violence and sexu...|Investigation com...|
|00157f2e34f43d43e...|        surrey|51.233094|-0.331669|        Public order|Formal action is ...|
|0019ed2657b030690...|cambridgeshire|52.186771| 0.188091|Theft from the pe...|Investigation com...|
|001be4a23bc23ed2c...|     hampshire| 50.85044|-1.252526|        Public order|Action to be take...|
|001c0cacca6991b60...|          kent|51.361226| 1.432812|Violence and sexu...|Investigation com...|


###### 1. Provide statistic

In [105]:
K = fin.groupBy("crimeType") \
       .agg(count("crimeID").alias("KPI")) \
       .where("KPI > 1") \
       .orderBy("KPI") \
       .sort("KPI", ascending=False)

In [106]:
K.show(60)

+--------------------+-----+
|           crimeType|  KPI|
+--------------------+-----+
|Violence and sexu...|57702|
|Criminal damage a...|21094|
|         Other theft|18973|
|        Public order|18039|
|       Vehicle crime|17929|
|         Shoplifting|11943|
|            Burglary| 7927|
|               Drugs| 6204|
|Theft from the pe...| 3700|
|         Other crime| 3143|
|       Bicycle theft| 3079|
|Possession of wea...| 1265|
|             Robbery|  851|
+--------------------+-----+



In [107]:
KPI = fin.groupBy("crimeType") \
       .agg(count("crimeID").alias("KPI")) \
       .orderBy("KPI") 

In [108]:
KPI2 = fin.groupBy("lastOutcomeCategory") \
       .agg(count("crimeID").alias("KPI")) \
       .orderBy("KPI") 

In [109]:
KPI3 = fin.groupBy("districtName") \
       .agg(count("crimeID").alias("KPI")) \
       .orderBy("KPI") 

In [29]:
KPI.show(50)
KPI2.show(50)
KPI3.show(50)

+--------------------+-----+
|           crimeType|  KPI|
+--------------------+-----+
|             Robbery|  886|
|Possession of wea...| 1302|
|       Bicycle theft| 3152|
|         Other crime| 3199|
|Theft from the pe...| 3884|
|               Drugs| 6414|
|            Burglary| 8063|
|         Shoplifting|12139|
|       Vehicle crime|18286|
|        Public order|18333|
|         Other theft|19405|
|Criminal damage a...|21388|
|Violence and sexu...|58656|
+--------------------+-----+



+--------------------+-----+
| lastOutcomeCategory|  KPI|
+--------------------+-----+
|Suspect charged a...|   27|
|Offender given a ...|  253|
|Offender given pe...|  507|
|Formal action is ...|  763|
| Under investigation|  782|
|Further action is...| 1192|
|Further investiga...| 1562|
|Offender given a ...| 1647|
|Action to be take...| 2680|
|    Local resolution| 5046|
|Awaiting court ou...|11051|
|Unable to prosecu...|54851|
|Investigation com...|94746|
+--------------------+-----+



+----------------+-----+
|    districtName|  KPI|
+----------------+-----+
|            city|  180|
|           gwent|  327|
|       wiltshire| 1117|
|         cumbria| 1658|
|           devon| 1681|
| gloucestershire| 1686|
|    warwickshire| 1687|
|          dorset| 1848|
|          durham| 1893|
|           dyfed| 1920|
|   hertfordshire| 2083|
|         suffolk| 2154|
|    bedfordshire| 2203|
|northamptonshire| 2777|
|         norfolk| 2904|
|           north| 3187|
|        cheshire| 3196|
|    lincolnshire| 3204|
|            avon| 3253|
|      derbyshire| 3253|
|  cambridgeshire| 3308|
|          surrey| 3361|
|          sussex| 3760|
|       cleveland| 4078|
| nottinghamshire| 4602|
|       hampshire| 4676|
|      lancashire| 4866|
|  leicestershire| 4958|
|     northumbria| 5033|
|          thames| 5854|
|      humberside| 5909|
|           essex| 8315|
|            kent|10268|
|      merseyside|10644|
|           south|12152|
|            west|17441|
|    metropolitan|23671|
